In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.layers.python.layers.initializers import variance_scaling_initializer

import skimage.io as io
import numpy as np
from src.util import image as img_util
from matplotlib import pyplot as plt

import deepdish as dd
from os.path import join, dirname

from utilsTest import *

import os.path as osp
import os
import sys
curr_path = osp.dirname(os.getcwd())
model_dir = osp.join(curr_path, '..', 'models')

smpl_model_path = r'C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\models\neutral_smpl_with_cocoplus_reg.pkl'
smpl_face_path = r'C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\src\tf_smpl\smpl_faces.npy'

from src.tf_smpl.batch_lbs import batch_rodrigues
from src.tf_smpl.batch_smpl import SMPL
from src.tf_smpl.projection import batch_orth_proj_idrot

In [2]:
img_path = 'data/im1963.jpg'
json_path = None
input_img, proc_param, img = preprocess_image(img_path, json_path)
input_img = np.expand_dims(input_img, 0)
input_img = np.float32(input_img)

Resizing so the max image size is 224..


In [3]:
def load_mean_param():
    mean = np.zeros((1, 85))
    # Initialize scale at 0.9
    mean[0, 0] = 0.9
    mean_path = join(
        dirname(smpl_model_path), 'neutral_smpl_mean_params.h5')
    mean_vals = dd.io.load(mean_path)

    mean_pose = mean_vals['pose']
    # Ignore the global rotation.
    mean_pose[:3] = 0.
    mean_shape = mean_vals['shape']

    # This initializes the global pose to be up-right when projected
    mean_pose[0] = np.pi

    mean[0, 3:] = np.hstack((mean_pose, mean_shape))
    mean = tf.constant(mean, tf.float32)
    mean_var = tf.Variable(
        mean, name="mean_param", dtype=tf.float32, trainable=True)
    E_var.append(mean_var)
    init_mean = tf.tile(mean_var, [1, 1])
    return init_mean

In [4]:
def Encoder_fc3_dropout(x,
                        num_output=85,
                        is_training=True,
                        reuse=False,
                        name="3D_module"):
    """
    3D inference module. 3 MLP layers (last is the output)
    With dropout  on first 2.
    Input:
    - x: N x [|img_feat|, |3D_param|]
    - reuse: bool

    Outputs:
    - 3D params: N x num_output
      if orthogonal: 
           either 85: (3 + 24*3 + 10) or 109 (3 + 24*4 + 10) for factored axis-angle representation
      if perspective:
          86: (f, tx, ty, tz) + 24*3 + 10, or 110 for factored axis-angle.
    - variables: tf variables
    """
    if reuse:
        print('Reuse is on!')
    with tf.variable_scope(name, reuse=reuse) as scope:
        net = slim.fully_connected(x, 1024, scope='fc1')
        net = slim.dropout(net, 0.5, is_training=is_training, scope='dropout1')
        net = slim.fully_connected(net, 1024, scope='fc2')
        net = slim.dropout(net, 0.5, is_training=is_training, scope='dropout2')
        small_xavier = variance_scaling_initializer(
            factor=.01, mode='FAN_AVG', uniform=True)
        net = slim.fully_connected(
            net,
            num_output,
            activation_fn=None,
            weights_initializer=small_xavier,
            scope='fc3')

    variables = tf.contrib.framework.get_variables(scope)
    return net, variables

In [5]:
use_3d_label = True
num_stage = 3
total_params = 85
num_cam = 3
num_theta = 72
batch_size = 1

In [6]:
# img_enc_fn, threed_enc_fn = get_encoder_fn_separate('resnet_fc3_dropout')
# img_feat, E_var = img_enc_fn(input_img, is_training=False, reuse=False)
img_feat, E_var = Encoder_resnet(input_img, is_training=False, reuse=False)
smpl = SMPL(smpl_model_path)

loss_kps = []
if use_3d_label:
    loss_3d_joints, loss_3d_params = [], []
# For discriminator
fake_rotations, fake_shapes = [], []
# Start loop
# 85D
theta_prev = load_mean_param()

# For visualizations
all_verts = []
all_pred_kps = []
all_pred_cams = []
all_delta_thetas = []
all_theta_prev = []

# Main IEF loop
for i in np.arange(num_stage):
    print('Iteration %d' % i)
    # ---- Compute outputs
    state = tf.concat([img_feat, theta_prev], 1)

    if i == 0:
#         delta_theta, _ = threed_enc_fn(state, num_output= total_params, reuse=False)
        delta_theta, _ = Encoder_fc3_dropout(state, num_output= total_params, reuse=False)
    else:
#         delta_theta, _ = threed_enc_fn(state, num_output= total_params, reuse=True)
        delta_theta, _ = Encoder_fc3_dropout(state, num_output= total_params, reuse=True)

    # Compute new theta
    theta_here = theta_prev + delta_theta
    # cam = N x 3, pose N x self.num_theta, shape: N x 10
    cams = theta_here[:, :num_cam]
    poses = theta_here[:, num_cam:(num_cam + num_theta)]
    shapes = theta_here[:, (num_cam + num_theta):]
    # Rs_wglobal is Nx24x3x3 rotation matrices of poses
    verts, Js, pred_Rs = smpl(shapes, poses, get_skin=True)
    pred_kp = batch_orth_proj_idrot(Js, cams, name='proj2d_stage%d' % i)
    # --- Compute losses:
#     loss_kps.append(keypoint_loss(
#         self.kp_loader, pred_kp))
    pred_Rs = tf.reshape(pred_Rs, [-1, 24, 9])
#     if self.use_3d_label:
#         loss_poseshape, loss_joints = self.get_3d_loss(
#             pred_Rs, shapes, Js)
#         loss_3d_params.append(loss_poseshape)
#         loss_3d_joints.append(loss_joints)

    # Save pred_rotations for Discriminator
    fake_rotations.append(pred_Rs[:, 1:, :])
    fake_shapes.append(shapes)

    # Finally update to end iteration.
    theta_prev = theta_here

Iteration 0


W0227 23:30:37.343826 16076 deprecation.py:323] From C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR\TensorFlow\src\tf_smpl\batch_lbs.py:55: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Iteration 1
Reuse is on!
Iteration 2
Reuse is on!


In [7]:
fake_rotations

[<tf.Tensor 'strided_slice_3:0' shape=(1, 23, 9) dtype=float32>,
 <tf.Tensor 'strided_slice_7:0' shape=(1, 23, 9) dtype=float32>,
 <tf.Tensor 'strided_slice_11:0' shape=(1, 23, 9) dtype=float32>]

In [8]:
fake_shapes

[<tf.Tensor 'strided_slice_2:0' shape=(1, 10) dtype=float32>,
 <tf.Tensor 'strided_slice_6:0' shape=(1, 10) dtype=float32>,
 <tf.Tensor 'strided_slice_10:0' shape=(1, 10) dtype=float32>]